<a href="https://colab.research.google.com/github/danfouer/Real-Time-Voice-Cloning/blob/master/HFAudio_Unit7_Hands_onexercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install --upgrade --quiet pip
!pip install --quiet git+https://github.com/huggingface/transformers.git
!pip install --upgrade accelerate
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 18.1 MB/s eta 0:00:00
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
import torch
from transformers import pipeline
from datasets import load_dataset
from IPython.display import Audio
from transformers import BarkModel
from transformers import AutoProcessor
import numpy as np

In [ ]:
!nvidia-smi

Sun Oct  8 01:05:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
notebook_login()

# Speech translation

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#device="cpu"
pipe = pipeline(
    "automatic-speech-recognition", model="openai/whisper-large-v2", device=device
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:


dataset = load_dataset("facebook/voxpopuli", "en", split="validation", streaming=True)
sample = next(iter(dataset))
print(sample)

{'audio_id': '20160526-0900-PLENARY-3-en_20160526-10:18:50_2', 'language': 0, 'audio': {'path': 'dev_part_0/20160526-0900-PLENARY-3-en_20160526-10:18:50_2.wav', 'array': array([-0.00088501, -0.0007019 , -0.00057983, ...,  0.00085449,
        0.00067139,  0.00061035]), 'sampling_rate': 16000}, 'raw_text': 'Many measures have been taken, not only in September but also in March, and of course we see some effects of those measures perhaps not enough, but there are effects of those measures, and the situation could have been worse if we did not have taken those measures.', 'normalized_text': 'many measures have been taken not only in september but also in march and of course we see some effects of those measures perhaps not enough but there are effects of those measures and the situation could have been worse if we did not have taken those measures.', 'gender': 'male', 'speaker_id': 'None', 'is_gold_transcript': True, 'accent': 'None'}


In [ ]:
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
def translate(audio):
    outputs = pipe(audio, max_new_tokens=256, generate_kwargs={"task": "transcribe","language":"chinese"})
    return outputs["text"]

In [ ]:
translate(sample["audio"].copy())

'很多措施在9月和3月中都被採取。當然,我們看到一些措施的影響,也許不足夠,但這些措施的影響確實存在,如果沒有採取這些措施,情況可能會更糟糕。'

# Text-to-speech

In [ ]:
model = BarkModel.from_pretrained("suno/bark-small")
processor = AutoProcessor.from_pretrained("suno/bark")

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
def synthesise(text_prompt,voice_preset="v2/zh_speaker_1"):
    inputs = processor(text_prompt, voice_preset=voice_preset)
    speech_output = model.generate(**inputs.to(device))
    synthesised_rate = model.generation_config.sample_rate
    return synthesised_rate,speech_output

In [ ]:
target_dtype = np.int16
max_range = np.iinfo(target_dtype).max


def speech_to_speech_translation(audio,voice_preset="v2/zh_speaker_1"):
    translated_text = translate(audio)
    synthesised_rate,synthesised_speech = synthesise(translated_text,voice_preset)
    synthesised_speech = (synthesised_speech.cpu().numpy() * max_range).astype(np.int16)
    return synthesised_rate,synthesised_speech

In [ ]:
synthesised_rate,synthesised_speech = speech_to_speech_translation(sample["audio"],"v2/zh_speaker_1")

Audio(synthesised_speech, rate=synthesised_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [ ]:
import gradio as gr

demo = gr.Blocks()

mic_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(source="microphone", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy"),
)

file_translate = gr.Interface(
    fn=speech_to_speech_translation,
    inputs=gr.Audio(source="upload", type="filepath"),
    outputs=gr.Audio(label="Generated Speech", type="numpy"),
)

with demo:
    gr.TabbedInterface([mic_translate, file_translate], ["Microphone", "Audio File"])

demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1033: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://91c4a7cf4152b669a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:188: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1563, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://91c4a7cf4152b669a7.gradio.live
